In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array

DATASET_PATH = "PetImages"
IMG_SIZE = (96, 96)
PHOTO_AMOUNT = 500

In [2]:
def load_data(path, img_size, sample_size):
    data = []
    labels = []
    classes = ['Dog', 'Cat']
    
    for label, class_name in enumerate(classes):
        class_path = os.path.join(path, class_name)
        images = os.listdir(class_path)[:sample_size]
        for img_name in images:
            img_path = os.path.join(class_path, img_name)
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img) / 255.0
            img_array = img_array.flatten()
            data.append(img_array)
            labels.append(label)
    
    return np.array(data), np.array(labels)

In [3]:
data, labels = load_data(DATASET_PATH, IMG_SIZE, PHOTO_AMOUNT)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'PetImages\\Dog'

In [ ]:
# Обучение базовой модели
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
base_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy без PCA: {base_accuracy:.4f}")

In [ ]:
# PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Число компонент для объяснения 90% дисперсии
explained_variance = np.cumsum(pca.explained_variance_ratio_)
n_components_90 = np.argmax(explained_variance >= 0.90) + 1
print(f"Число компонент для объяснения 90% дисперсии: {n_components_90}")

In [ ]:
# Обучение модели на данных с PCA
clf_pca = RandomForestClassifier(n_estimators=100, random_state=42)
clf_pca.fit(X_train_pca[:, :n_components_90], y_train)
y_pred_pca = clf_pca.predict(X_test_pca[:, :n_components_90])
pca_accuracy = accuracy_score(y_test, y_pred_pca)
print(f"Accuracy с PCA ({n_components_90} компонент): {pca_accuracy:.4f}")

In [ ]:
# График зависимости качества от числа компонент
component_range = np.arange(1, X_train_pca.shape[1] + 1, 10)
accuracies = []
for n in component_range:
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train_pca[:, :n], y_train)
    y_pred = clf.predict(X_test_pca[:, :n])
    accuracies.append(accuracy_score(y_test, y_pred))

plt.figure(figsize=(8, 5))
plt.plot(component_range, accuracies, marker='o')
plt.xlabel("Число компонент PCA")
plt.ylabel("Точность")
plt.title("Зависимость качества классификации от числа компонент")
plt.show()

# График зависимости объясненной дисперсии от числа компонент
plt.figure(figsize=(8, 5))
plt.plot(np.arange(1, len(explained_variance) + 1), explained_variance, marker='o')
plt.axhline(y=0.9, color='r', linestyle='--')
plt.xlabel("Число компонент")
plt.ylabel("Процент объясненной дисперсии")
plt.title("Зависимость объясненной дисперсии от числа компонент")
plt.show()

In [ ]:
# Первые 10 главных компонент
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.ravel()):
    component = pca.components_[i].reshape(IMG_SIZE[0], IMG_SIZE[1], 3)
    component = (component - component.min()) / (component.max() - component.min())  # Нормализация
    ax.imshow(component)
    ax.set_title(f'Компонента {i+1}')
    ax.axis('off')
plt.show()

In [ ]:
# Сравнение моделей
models = {
    "Логистическая регрессия": LogisticRegression(max_iter=1000),
    "Дерево решений": DecisionTreeClassifier(),
    "Случайный лес": RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, model in models.items():
    model.fit(X_train_pca[:, :n_components_90], y_train)
    y_pred = model.predict(X_test_pca[:, :n_components_90])
    print(f"Точность {name} с PCA: {accuracy_score(y_test, y_pred):.4f}")